In [1]:
import findspark
findspark.init()
import pyspark
from random import random
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark import sql
import itertools
import time
from pyspark.mllib.fpm import FPGrowth

In [2]:
import os
port = 4040 + hash(os.getcwd().split("/")[2])%200
config = pyspark.SparkConf().set('spark.executor.memory', '512M')\
         .set('spark.cores.max', '2')\
         .set('spark.port.maxRetries','200')\
         .set('spark.ui.port', port)
        

print port,     os.getcwd().split("/")[2]

4150 ajp218


In [3]:
sc = pyspark.SparkContext(appName="", master='',conf = config )

In [4]:
rddData = sc.textFile("data/recipes.txt")
print rddData.take(5)

[u'greek;10259;romaine lettuce;black olives;grape tomatoes;garlic;pepper;purple onion;seasoning;garbanzo beans;feta cheese crumbles', u'southern_us;25693;plain flour;ground pepper;salt;tomatoes;ground black pepper;thyme;eggs;green tomatoes;yellow corn meal;milk;vegetable oil', u'filipino;20130;eggs;pepper;salt;mayonaise;cooking oil;green chilies;grilled chicken breasts;garlic powder;yellow onion;soy sauce;butter;chicken livers', u'indian;22213;water;vegetable oil;wheat;salt', u'indian;13162;black pepper;shallots;cornflour;cayenne pepper;onions;garlic paste;milk;butter;salt;lemon juice;water;chili powder;passata;oil;ground cumin;boneless chicken skinless thigh;garam masala;double cream;natural yogurt;bay leaf']


In [5]:
def Apriori(data, minSupport, minItems):
  
 # define necessary functions
    
    def reducing_filter(data):
        check_val = [j[0] for j in check]

        if frequencyCount==2:
            data_combs = [i[0] for i in list(itertools.combinations(data, frequencyCount-1))]

        else:
            data_combs = [i for i in list(itertools.combinations(data, frequencyCount-1))]

        out = []

        for i in data_combs:
            if i in check_val:
                out += [(i)]

        if len(out) != 0 and frequencyCount==2:
            return list(itertools.combinations(out, frequencyCount))

        elif len(out) != 0:
            out = [list(i) for i in out]
            out = set(x for i in out for x in i)
            return list(itertools.combinations(out, frequencyCount))

        else:
            return None

 # get results
    result = []
    total_time = 0
    frequencyCount = 1

    while True:

        t1 = time.time()
        
        if frequencyCount==1:
            stage_res = data.flatMap(lambda x: x).map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y).cache()
            supportVal = minSupport*data.count()

        else: 
            stage_res = data.map(reducing_filter).filter(lambda x: x!=None and x!=[]).flatMap(lambda x: x).map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y).cache()

        check = stage_res.filter(lambda kv: kv[1] >= supportVal).collect()
        
        if frequencyCount >= minItems:
            result.extend(check)
        
        iteration_time = (time.time()-t1)/60
        total_time+=iteration_time
        print 'The runtime for item lists of length {:d} is {:6.4f} minutes'.format(frequencyCount, iteration_time)
        
        frequencyCount+=1
        
        if len(check) == 0:
            break
    
    print '\nThe total runtime is {:6.4f} minutes'.format(total_time)
        
    return sorted(result, key=lambda x: -x[1])

In [6]:
testData = rddData.map(lambda x: x.split(';')[2:]).map(lambda x: list(set(x)))
AprioriResult = Apriori(testData, 0.05, 1)

The runtime for item lists of length 1 is 0.0622 minutes
The runtime for item lists of length 2 is 0.0212 minutes
The runtime for item lists of length 3 is 0.0336 minutes

The total runtime is 0.1170 minutes


In [7]:
t1 = time.time()
FPmodel = FPGrowth.train(testData, minSupport=0.05, numPartitions=10)
FPresult = FPmodel.freqItemsets().sortBy(lambda x: x[1], False).collect()
print 'The total runtime for FPGrowth is {:6.4f} minutes'.format( (time.time()-t1)/60 )

The total runtime for FPGrowth is 0.0912 minutes


In [12]:
print 'The Apriori method is {:f} times slower than the FPGrowth method'.format(0.1170/0.0912)

The Apriori method is 1.282895 times slower than the FPGrowth method


In [9]:
it = 1
for i in range(0, len(AprioriResult)):
    print 'Apriori  #{:d}: {:s}: {:d}'.format(it, [AprioriResult[i][0]], AprioriResult[i][1])
    it+=1

Apriori  #1: [u'salt']: 18048
Apriori  #2: [u'onions']: 7972
Apriori  #3: [u'olive oil']: 7971
Apriori  #4: [u'water']: 7457
Apriori  #5: [u'garlic']: 7380
Apriori  #6: [u'sugar']: 6434
Apriori  #7: [u'garlic cloves']: 6236
Apriori  #8: [u'butter']: 4847
Apriori  #9: [u'ground black pepper']: 4784
Apriori  #10: [u'all-purpose flour']: 4632
Apriori  #11: [u'pepper']: 4438
Apriori  #12: [u'vegetable oil']: 4385
Apriori  #13: [(u'olive oil', u'salt')]: 4128
Apriori  #14: [(u'pepper', u'salt')]: 3748
Apriori  #15: [u'eggs']: 3388
Apriori  #16: [u'soy sauce']: 3296
Apriori  #17: [(u'water', u'salt')]: 3180
Apriori  #18: [u'kosher salt']: 3113
Apriori  #19: [u'green onions']: 3078
Apriori  #20: [u'tomatoes']: 3058
Apriori  #21: [u'large eggs']: 2948
Apriori  #22: [(u'garlic cloves', u'salt')]: 2917
Apriori  #23: [u'carrots']: 2814
Apriori  #24: [u'unsalted butter']: 2782
Apriori  #25: [u'ground cumin']: 2747
Apriori  #26: [u'extra-virgin olive oil']: 2747
Apriori  #27: [(u'butter', u'salt')]

In [10]:
it = 1
for i in range(0, len(FPresult)):
    print 'FPGrowth #{:d}: {:s}: {:d}'.format(it, FPresult[i][0], FPresult[i][1])
    it+=1

FPGrowth #1: [u'salt']: 18048
FPGrowth #2: [u'onions']: 7972
FPGrowth #3: [u'olive oil']: 7971
FPGrowth #4: [u'water']: 7457
FPGrowth #5: [u'garlic']: 7380
FPGrowth #6: [u'sugar']: 6434
FPGrowth #7: [u'garlic cloves']: 6236
FPGrowth #8: [u'butter']: 4847
FPGrowth #9: [u'ground black pepper']: 4784
FPGrowth #10: [u'all-purpose flour']: 4632
FPGrowth #11: [u'pepper']: 4438
FPGrowth #12: [u'onions', u'salt']: 4392
FPGrowth #13: [u'vegetable oil']: 4385
FPGrowth #14: [u'olive oil', u'salt']: 4177
FPGrowth #15: [u'water', u'salt']: 3960
FPGrowth #16: [u'pepper', u'salt']: 3844
FPGrowth #17: [u'garlic', u'salt']: 3749
FPGrowth #18: [u'eggs']: 3388
FPGrowth #19: [u'soy sauce']: 3296
FPGrowth #20: [u'kosher salt']: 3113
FPGrowth #21: [u'all-purpose flour', u'salt']: 3079
FPGrowth #22: [u'green onions']: 3078
FPGrowth #23: [u'sugar', u'salt']: 3061
FPGrowth #24: [u'tomatoes']: 3058
FPGrowth #25: [u'garlic cloves', u'salt']: 2995
FPGrowth #26: [u'large eggs']: 2948
FPGrowth #27: [u'carrots']: 28

In [11]:
it = 1
for i in range(0, len(AprioriResult)):
    print 'Apriori  #{:d}: {:s}: {:d}'.format(it, [AprioriResult[i][0]], AprioriResult[i][1])
    print 'FPGrowth #{:d}: {:s}: {:d}'.format(it, FPresult[i][0], FPresult[i][1])
    it+=1

Apriori  #1: [u'salt']: 18048
FPGrowth #1: [u'salt']: 18048
Apriori  #2: [u'onions']: 7972
FPGrowth #2: [u'onions']: 7972
Apriori  #3: [u'olive oil']: 7971
FPGrowth #3: [u'olive oil']: 7971
Apriori  #4: [u'water']: 7457
FPGrowth #4: [u'water']: 7457
Apriori  #5: [u'garlic']: 7380
FPGrowth #5: [u'garlic']: 7380
Apriori  #6: [u'sugar']: 6434
FPGrowth #6: [u'sugar']: 6434
Apriori  #7: [u'garlic cloves']: 6236
FPGrowth #7: [u'garlic cloves']: 6236
Apriori  #8: [u'butter']: 4847
FPGrowth #8: [u'butter']: 4847
Apriori  #9: [u'ground black pepper']: 4784
FPGrowth #9: [u'ground black pepper']: 4784
Apriori  #10: [u'all-purpose flour']: 4632
FPGrowth #10: [u'all-purpose flour']: 4632
Apriori  #11: [u'pepper']: 4438
FPGrowth #11: [u'pepper']: 4438
Apriori  #12: [u'vegetable oil']: 4385
FPGrowth #12: [u'onions', u'salt']: 4392
Apriori  #13: [(u'olive oil', u'salt')]: 4128
FPGrowth #13: [u'vegetable oil']: 4385
Apriori  #14: [(u'pepper', u'salt')]: 3748
FPGrowth #14: [u'olive oil', u'salt']: 4177


In [13]:
sc.stop()